then login and get a token from the huggingface user access tokens page:

In [11]:
#uncomment if you haven't already logged in.
#from huggingface_hub import login
#login()
# can also do this from login with hf auth login
# there is also notebook login
# from huggingface_hub import notebook_login
# notebook_login()
# this is for cloud notebooks. 

In [1]:
import transformers
import torch
import json, sys, os

The most basic usecase is the pipeline:

In [2]:
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Llama-3.1-8B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [3]:
messages = [
    {"role": "user", "content": "Who are you?"},
]

In [4]:
messages = pipe(messages)[0]['generated_text']
messages

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'role': 'user', 'content': 'Who are you?'},
 {'role': 'assistant',
  'content': 'I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."'}]

In [5]:
messages.append({'role': 'user', 'content': 'what is quantum mechanics?'})
messages = pipe(messages)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [6]:
messages

[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."'},
   {'role': 'user', 'content': 'what is quantum mechanics?'},
   {'role': 'assistant',
    'content': "Quantum mechanics is a branch of physics that studies the behavior of matter and energy at an atomic and subatomic level. At these scales, the classical laws of physics no longer apply, and strange, seemingly random phenomena govern the behavior of particles.\n\nSome key principles of quantum mechanics include:\n\n1. **Wave-particle duality**: Particles, such as electrons, can exhibit both wave-like and particle-like behavior.\n2. **Uncertainty principle**: It's impossible to know certain properties of a particle, like its position and momentum, simultaneously with infinite precision.\n3. **Superposition**: Particles can exist in multiple states simultaneously, like spinning 

Here is another interesting model:

In [2]:
from PIL import Image
from transformers import AutoTokenizer, AutoProcessor, AutoModelForImageTextToText

model_path = "nanonets/Nanonets-OCR2-3B"

model = AutoModelForImageTextToText.from_pretrained(
    model_path, 
    torch_dtype="auto", 
    device_map="auto", 
    attn_implementation="sdpa"
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_path)
processor = AutoProcessor.from_pretrained(model_path)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def ocr_page_with_nanonets_s(image_path, model, processor, max_new_tokens=4096):
    prompt = """Extract the text from the above document as if you were reading it naturally. Return the tables in html format. Return the equations in LaTeX representation. If there is an image in the document and image caption is not present, add a small description of the image inside the <img></img> tag; otherwise, add the image caption inside <img></img>. Watermarks should be wrapped in brackets. Ex: <watermark>OFFICIAL COPY</watermark>. Page numbers should be wrapped in brackets. Ex: <page_number>14</page_number> or <page_number>9/22</page_number>. Prefer using ☐ and ☑ for check boxes."""
    image = Image.open(image_path)
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": [
            {"type": "image", "image": f"file://{image_path}"},
            {"type": "text", "text": prompt},
        ]},
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[image], padding=True, return_tensors="pt")
    inputs = inputs.to(model.device)
    
    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output_ids)]
    
    output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return output_text[0]

![blah](random_eq.jpeg)

In [4]:
out = ocr_page_with_nanonets_s('/home/louis/research/hf_demo/random_eq.jpeg', model, processor)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [5]:
out

'<header>Scattering Region WKB</header>\n\nIn the scattering region we have\n\n$$\\begin{bmatrix} \\Delta & P(\\frac{\\partial}{\\partial x}, k_y) \\\\ P^*(\\frac{\\partial}{\\partial x}, k_y) & -\\Delta \\end{bmatrix}\\begin{bmatrix} u(x) \\\\ v(x) \\end{bmatrix} + V(x)\\begin{bmatrix} u(x) \\\\ v(x) \\end{bmatrix} = E\\begin{bmatrix} u(x) \\\\ v(x) \\end{bmatrix}$$\n\nwriting it out\n\n$$\\left(A_{xx}\\frac{\\partial^2}{\\partial x^2} + A_x\\frac{\\partial}{\\partial x} + A_0\\right)v(x) + \\left(\\Delta + V(x)\\right)u(x) = Eu(x)$$\n\n$$\\left(A_{xx}^*\\frac{\\partial^2}{\\partial x^2} + A_x^*\\frac{\\partial}{\\partial x} + A_0^*\\right)u(x) + \\left(-\\Delta + V(x)\\right)v(x) = Ev(x)$$\n\n$$\\frac{\\partial^2v}{\\partial x^2} = -\\frac{A_x}{A_{xx}}\\frac{\\partial v}{\\partial x} - \\frac{A_0}{A_{xx}}v - \\frac{1}{A_{xx}}(\\Delta + V(x) - E)u(x) = A\\frac{\\partial v}{\\partial x} + B(E)u + Fv$$\n\n$$\\frac{\\partial^2u}{\\partial x^2} = -\\frac{A_x^*}{A_{xx}^*}\\frac{\\partial u

\n\nIn the scattering region we have\n\n$$\begin{bmatrix} \Delta & P(\frac{\partial}{\partial x}, k_y) \\ P^*(\frac{\partial}{\partial x}, k_y) & -\Delta \end{bmatrix}\begin{bmatrix} u(x) \\ v(x) \end{bmatrix} + V(x)\begin{bmatrix} u(x) \\ v(x) \end{bmatrix} = E\begin{bmatrix} u(x) \ v(x) \end{bmatrix}$$\n\nwriting it out\n\n$$\left(A_{xx}\frac{\partial^2}{\partial x^2} + A_x\frac{\partial}{\partial x} + A_0\right)v(x) + \left(\Delta + V(x)\right)u(x) = Eu(x)$$\n\n$$\left(A_{xx}^*\frac{\partial^2}{\partial x^2} + A_x^*\frac{\partial}{\partial x} + A_0^*\right)u(x) + \left(-\Delta + V(x)\right)v(x) = Ev(x)$$\n\n$$\frac{\partial^2v}{\partial x^2} = -\frac{A_x}{A_{xx}}\frac{\partial v}{\partial x} - \frac{A_0}{A_{xx}}v - \frac{1}{A_{xx}}(\Delta + V(x) - E)u(x) = A\frac{\partial v}{\partial x} + B(E)u + Fv$$\n\n$$\frac{\partial^2u}{\partial x^2} = -\frac{A_x^*}{A_{xx}^*}\frac{\partial u}{\partial x} - \frac{A_0^*}{A_{xx}^*}u - \frac{1}{A_{xx}^*}(-\Delta + V(x) - E)v(x) = A^*\frac{\partial u}{\partial x} + C(E)v + F^*u$$